In [16]:
import sys
sys.path.append('d:\\GitHub\\Mini-Projects\\LLM\\config')

from config import googleAPIKey
import re

with open('/GitHub/Mini-Projects/LLM/config/config.py', 'r') as databaseUrlFile:
    googleAPIKey = databaseUrlFile.read()

match = re.search(r'"([^"]*)"', googleAPIKey)

if match:
    googleAPIKey = match.group(1)
else:
    print("No match found")

In [20]:
from langchain.llms import GooglePalm

# temperature - control creativity level
llm = GooglePalm(google_api_key=googleAPIKey,temperature=0.7)

In [21]:
poem = llm("write a 4 line of my love for samosa")
print(poem)

**Samosas, oh samosas,**
**You are the food of my dreams.**
**Your crispy shell and savory filling**
**Fill me with joy and glee.**


In [24]:
essay = llm("write email requesting refund for electronic item")
print(essay)

```
Dear [Company Name],

I am writing to request a refund for the [product name] that I purchased on [date]. I am not satisfied with the product because [reason].

I have attached a copy of my receipt and a picture of the product. I have also included a description of the problem that I am having with the product.

I would appreciate it if you would process my refund as soon as possible. I would like to be refunded the full amount that I paid for the product.

Thank you for your time and consideration.

Sincerely,
[Your Name]
```


## Langchain CSV loader

In [29]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="codebasics_faqs.csv", source_column='prompt')
data= loader.load()

In [42]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS

# TODO - Learn Embeddings basics
instructor_embeddings = HuggingFaceInstructEmbeddings()
vectordb = FAISS.from_documents(documents=data, embedding=instructor_embeddings)


load INSTRUCTOR_Transformer
max_seq_length  512


In [44]:
# https://python.langchain.com/docs/modules/data_connection/retrievers/vectorstore#maximum-marginal-relevance-retrieval
retriever = vectordb.as_retriever()
relevantDocs = retriever.get_relevant_documents("for how long is this course valid?")

relevantDocs

[Document(page_content='prompt: Once purchased, is this course available for lifetime access?\nresponse: Yes', metadata={'source': 'Once purchased, is this course available for lifetime access?', 'row': 22}),
 Document(page_content='prompt: What is the duration of this bootcamp? How long will it last?\nresponse: You can complete all courses in 3 months if you dedicate 2-3 hours per day.', metadata={'source': 'What is the duration of this bootcamp? How long will it last?', 'row': 8}),
 Document(page_content='prompt: Will the course be upgraded when there are new features in Power BI?\nresponse: Yes, the course will be upgraded periodically based on the new features in Power BI, and learners who have already bought this course will have free access to the upgrades.', metadata={'source': 'Will the course be upgraded when there are new features in Power BI?', 'row': 27}),
 Document(page_content='prompt: Is this bootcamp enough for me in Microsoft Power BI and\n Excel certifications?\nrespo

In [46]:
from langchain.chains import RetrievalQA 

chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, input_key="query", return_source_documents=True)

In [47]:
chain("Do you guys provide internship and also do you offer EMI payments?")

{'query': 'Do you guys provide internship and also do you offer EMI payments?',
 'result': "Yes, we provide virtual internship and no, we don't offer EMI payments.",
 'source_documents': [Document(page_content='prompt: Do you provide any virtual internship?\nresponse: Yes', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}),
  Document(page_content='prompt: Do we have an EMI option?\nresponse: No', metadata={'source': 'Do we have an EMI option?', 'row': 13}),
  Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
  Document(page_content='prompt: How can I contact the instructors for any doubts/support?\nresponse: We have created every lecture with a motive to explain everything in an e

In [49]:
# creates its own answer
chain("Should I learn power bi or tableau?")

{'query': 'Should I learn power bi or tableau?',
 'result': 'Power BI is cheaper and offers tighter integration with the Microsoft environment. Since most companies use excel & Microsoft tools they start with Power BI or move towards Power BI for seamless integration with other Microsoft tools (called as Power platform). This makes the job openings grow at a much higher rate on Power BI and Power Platform. Also, Power BI has been leading the Gartner’s magic quadrant in BI for the last few years as the industry leader.',
 'source_documents': [Document(page_content='prompt: Power BI or Tableau which one is better?\nresponse: This is a contextual question. If you are talking about a pure visualization tool Tableau is slightly better. Data connectors, modeling and transformation features are available in both. However, factually speaking Power BI is cheaper and offers tighter integration with the Microsoft environment. Since most companies use excel & Microsoft tools they start with Power 

In [55]:
# hallucinates
chain("do you have javascript course?")

{'query': 'do you have javascript course?',
 'result': 'no',
 'source_documents': [Document(page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.', metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}),
  Document(page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.', metadata={'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0}),
  Document(

In [56]:
from langchain.prompts import PromptTemplate

In [60]:
# Reduce hallucination
prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""

PROMPT = PromptTemplate(
template = prompt_template,input_variables=["context","question"]
)
chain_type_kwargs = {"prompt": PROMPT}

chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    input_key="query", 
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [61]:
chain("do you have javascript course?")

{'query': 'do you have javascript course?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.', metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}),
  Document(page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.', metadata={'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0}),
